In [1]:
import json
import os
import pandas
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

/Users/victoria/GitHub/venvs/bespoke/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
metrc_download_summary_companies_query = create_queries.create_metrc_download_summary_companies_query()
metrc_download_summary_companies_dataframe = pandas.read_sql_query(metrc_download_summary_companies_query, engine)

In [19]:
metrc_download_summary_companies_dataframe['identifier'].unique()

array(['1346', '5MIL', '64HM', None, '99HT', 'AGA', 'AHCOM', 'ALT',
       'Local Roots', 'AL', 'BMG', 'BFG', 'BS', 'BMC', 'BT', 'LEG', 'BBF',
       'BDS', 'BD', 'BUD', 'BRC', 'BYN', 'CHO', 'CG', 'CPA', 'CPC', 'CSC',
       'CSCC', 'CCC', 'CLC', 'CC', 'CED', 'CRRH', 'LR', 'CAT', 'CE', 'CI',
       'CTE', 'CDCA', 'CH', 'CBF', 'DHST', 'DWF', 'DEI', 'DM', 'DIV1',
       'DDF', 'DCO', 'DL', 'DFC', 'DFM', 'DP', 'DGG', 'DG', 'EJ', 'EL',
       'EMA', 'EMB', 'EMF', 'EMM', 'EMS', 'EMT', 'EPC', 'EH', 'ETR', 'FI',
       'FSM', 'FBCO', 'FF', 'FW', 'GB2', 'GGMA', 'GLO', 'GTC', 'GT',
       'GDN', 'GDS', 'GLNR', 'GCW', 'GL', 'GMFL', 'GRG', 'GTR', 'GHC',
       'GP', 'GPF', 'GF', 'GFEE', 'GFWF', 'GBM', 'HTC', 'HDEX', 'HB',
       'HBL', 'HS', 'HGD', 'HGH', 'HDVW', 'HPCC', 'HC', 'HDC', 'THL',
       'IDC', 'ID', 'WVN', 'JC', 'KAR', 'KC', 'KSS', 'LBF', 'LEGP', 'LLL',
       'LBC', 'LL', 'MCT', 'MSS', 'ML', 'MW', 'MPW', 'MLH', 'MD', 'MR',
       'MSBOR', 'MV', 'NWCCA', 'ND', 'NATD', 'NECC', 'NT', 'NI

In [21]:
metrc_download_summary_companies_dataframe = pandas.read_csv('retail.csv')

In [22]:
company_records = metrc_download_summary_companies_dataframe.to_dict('records')

In [23]:
company_records

[{'Unnamed: 0': 0.0,
  'name': '1346, LLC',
  'identifier': '1346',
  'id': 'b3405915-c319-448b-a16f-65b57f57f951'},
 {'Unnamed: 0': 1.0,
  'name': '5MIL',
  'identifier': '5MIL',
  'id': '0e42f66d-45c4-4a0d-bb8e-ba65b1c57222'},
 {'Unnamed: 0': 2.0,
  'name': '64 and Hope Melrose',
  'identifier': '64HM',
  'id': '078a3d8c-ebaf-4c0a-8ed6-ac4326df9993'},
 {'Unnamed: 0': 4.0,
  'name': '99 High Tide',
  'identifier': '99HT',
  'id': '54013296-4d58-48bb-bd1b-a6e7b2d17aa3'},
 {'Unnamed: 0': 5.0,
  'name': 'A Green Alternative Inc.',
  'identifier': 'AGA',
  'id': 'f9c729db-66c8-4eae-9dd1-40a988ec8862'},
 {'Unnamed: 0': 6.0,
  'name': 'AHCOM MGMT CO LLC',
  'identifier': 'AHCOM',
  'id': '5ee0c3fa-a1c6-4a18-a575-89d81ddcc6fa'},
 {'Unnamed: 0': 9.0,
  'name': 'Aureum Labs',
  'identifier': 'AL',
  'id': 'ddc07d90-aac6-4af0-ae2e-c7b9d0ad9c59'},
 {'Unnamed: 0': 7.0,
  'name': 'Altum LLC',
  'identifier': 'ALT',
  'id': 'c894d55e-3af1-444a-9941-b7649ceb5efd'},
 {'Unnamed: 0': 17.0,
  'name': 'B

In [24]:
company_to_retailer_licenses = {}
company_identifier_to_us_state = {}

for company_record in company_records:
    company_identifier = company_record['identifier']
    company_name = company_record['name']

    key = (company_identifier, company_name)
    
    print(f'Checking if company {company_name} ({company_identifier}) holds any retailer licenses...')
    company_licenses_query = create_queries.create_company_licenses_query(company_identifier)
    company_licenses_dataframe = pandas.read_sql_query(company_licenses_query, engine)
    company_license_records = company_licenses_dataframe.to_dict('records')
    for company_license_record in company_license_records:
        us_state = company_license_record['us_state']
        company_identifier_to_us_state[company_identifier] = us_state

        license_number = company_license_record['license_number']
        license_category = company_license_record['license_category']
        if license_category in ['Multiple', 'Retailer']:
            if key not in company_to_retailer_licenses:
                company_to_retailer_licenses[key] = []
            company_to_retailer_licenses[key].append(license_number)

    print(f'Checking if company {company_name} ({company_identifier}) has any sales receipts...')
    company_count_metrc_sales_receipts_query = create_queries.create_company_count_metrc_sales_receipts_query(company_identifier)
    company_count_metrc_sales_receipts_dataframe = pandas.read_sql_query(company_count_metrc_sales_receipts_query, engine)
    
    has_sales_receipts = len(company_count_metrc_sales_receipts_dataframe.index) > 0
    if has_sales_receipts:
        company_count_metrc_sales_receipt_record = company_count_metrc_sales_receipts_dataframe.to_dict('records')[0]
        sales_receipts_count = company_count_metrc_sales_receipt_record['count']
        if key not in company_to_retailer_licenses:
            print(f'WARNING!!!')
            print(f'WARNING: found company {company_name} ({company_identifier}) with {sales_receipts_count} sales receipts but no retailer license!')
    else:
        if key in company_to_retailer_licenses:
            print(f'WARNING!!!')
            print(f'WARNING: found company {company_name} ({company_identifier}) with 0 sales receipts but with retailer license(s)!')
            del company_to_retailer_licenses[key]

Checking if company 1346, LLC (1346) holds any retailer licenses...
Checking if company 1346, LLC (1346) has any sales receipts...
Checking if company 5MIL (5MIL) holds any retailer licenses...
Checking if company 5MIL (5MIL) has any sales receipts...
Checking if company 64 and Hope Melrose (64HM) holds any retailer licenses...
Checking if company 64 and Hope Melrose (64HM) has any sales receipts...
Checking if company 99 High Tide (99HT) holds any retailer licenses...
Checking if company 99 High Tide (99HT) has any sales receipts...
Checking if company A Green Alternative Inc. (AGA) holds any retailer licenses...
Checking if company A Green Alternative Inc. (AGA) has any sales receipts...
Checking if company AHCOM MGMT CO LLC (AHCOM) holds any retailer licenses...
Checking if company AHCOM MGMT CO LLC (AHCOM) has any sales receipts...
Checking if company Aureum Labs (AL) holds any retailer licenses...
Checking if company Aureum Labs (AL) has any sales receipts...
Checking if company A

Checking if company Embarc Alameda (EMA) has any sales receipts...
Checking if company Embarc Blackstone (EMB) holds any retailer licenses...
Checking if company Embarc Blackstone (EMB) has any sales receipts...
Checking if company Embarc Fairfield (EMF) holds any retailer licenses...
Checking if company Embarc Fairfield (EMF) has any sales receipts...
Checking if company Embarc Martinez (EMM) holds any retailer licenses...
Checking if company Embarc Martinez (EMM) has any sales receipts...
Checking if company Embarc Sacramento (EMS) holds any retailer licenses...
Checking if company Embarc Sacramento (EMS) has any sales receipts...
Checking if company Embarc Tahoe (EMT) holds any retailer licenses...
Checking if company Embarc Tahoe (EMT) has any sales receipts...
Checking if company Emerald Perspective Cooperative, Inc (EPC) holds any retailer licenses...
Checking if company Emerald Perspective Cooperative, Inc (EPC) has any sales receipts...
Checking if company Eternabis Inc (ETR) h

Checking if company Ladybug Farms (LBF) holds any retailer licenses...
Checking if company Ladybug Farms (LBF) has any sales receipts...
Checking if company Black Market (Lake Effect Group) (LEG) holds any retailer licenses...
Checking if company Black Market (Lake Effect Group) (LEG) has any sales receipts...
Checking if company Lake Effect - Portage (LEGP) holds any retailer licenses...
Checking if company Lake Effect - Portage (LEGP) has any sales receipts...
Checking if company Luminescence Labs (LL) holds any retailer licenses...
Checking if company Luminescence Labs (LL) has any sales receipts...
Checking if company Lion Labs (LLL) holds any retailer licenses...
Checking if company Lion Labs (LLL) has any sales receipts...
Checking if company Archived (Local Roots) holds any retailer licenses...
Checking if company Archived (Local Roots) has any sales receipts...
WARNING!!!
Checking if company Caregiver Patient Connection, LLC (LR) holds any retailer licenses...
Checking if compa

Checking if company Sitka Northwest, LLC. (SNW) holds any retailer licenses...
Checking if company Sitka Northwest, LLC. (SNW) has any sales receipts...
Checking if company Sovereign (SO) holds any retailer licenses...
Checking if company Sovereign (SO) has any sales receipts...
Checking if company Sugars Refinery (SR) holds any retailer licenses...
Checking if company Sugars Refinery (SR) has any sales receipts...
Checking if company Sticky Thumb (ST) holds any retailer licenses...
Checking if company Sticky Thumb (ST) has any sales receipts...
Checking if company Sava (SV) holds any retailer licenses...
Checking if company Sava (SV) has any sales receipts...
Checking if company Tree4Life (T4L) holds any retailer licenses...
Checking if company Tree4Life (T4L) has any sales receipts...
Checking if company The Good Leaf (TGL) holds any retailer licenses...
Checking if company The Good Leaf (TGL) has any sales receipts...
Checking if company Humboldt Leaf (THL) holds any retailer licens

TypeError: 'float' object is not iterable

In [ ]:
# for company_tuple, retailer_licenses in company_to_retailer_licenses.items():
#     company_identifier, company_name = company_tuple
#     print(f'Company {company_name} ({company_identifier}) holds Retailer licenses: {retailer_licenses}')

In [25]:
len(company_to_retailer_licenses.keys())

115

In [26]:
# Output to copy to XLSX
# Columns: company name, company identifier, US state, semi-colon separated license numbers
retailer_company_tuples = []

for company_tuple, retailer_licenses in company_to_retailer_licenses.items():
    company_identifier, company_name = company_tuple
    company_licenses = ";".join(retailer_licenses)
    us_state = company_identifier_to_us_state[company_identifier]
#     print((company_name, company_identifier, us_state, retailer_licenses))
    retailer_company_tuples.append((company_name, company_identifier, us_state, company_licenses))

In [27]:
columns = [
    'Company name',
    'Company identifier',
    'US state',
    'Company licenses',
]
retailer_companies_dataframe = pandas.DataFrame(retailer_company_tuples, columns=columns)
retailer_companies_dataframe

,Company name,Company identifier,US state,Company licenses
0,64 and Hope Melrose,64HM,CA,C10-0000922-LIC
1,99 High Tide,99HT,CA,C10-0000279-LIC
2,A Green Alternative Inc.,AGA,CA,C10-0000341-LIC
3,Altum LLC,ALT,MI,PC-000310;AU-R-000380
4,Boston Bud Factory,BBF,MA,MR281525
...,...,...,...,...
110,Urbn Leaf,UR,CA,C10-0000776-LIC;C10-0000246-LIC;C10-0000731-LI...
111,Voyage,VOY,CA,C12-0000030-LIC;C10-0000802-LIC;C12-0000159-LIC
112,Verts,VS,CO,402-00251;402R-00545
113,Palm Desert Melanie Place Industrial,WCCC,CA,C10-0000045-LIC;C10-0000372-LIC;C12-0000119-LIC


In [28]:
retailer_companies_dataframe.to_csv('dec_test.csv')

In [ ]:
import time
from datetime import date

TODAY_DATE = date.today().strftime('%m-%d-%Y')
NOW = int(time.time())

retailer_companies_file_name = f'~/Downloads/retailer_companies_file_name_{TODAY_DATE}_{NOW}'

num_retailer_companies = len(retailer_companies_dataframe.index)
print(f'Exporting {num_retailer_companies} retailer companies to files...')
retailer_companies_dataframe.reset_index().to_excel(f'{retailer_companies_file_name}.xlsx')
# export_incoming_transfer_packages_dataframe.reset_index().to_csv(f'{incoming_transfer_packages_file_name}.csv')
print(f'Exported {num_retailer_companies} retailer companies to files')